In [1]:
import seaborn as sns
import polars as pl

In [4]:
tip_df = sns.load_dataset('tips')
tip_df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [6]:
fli_df = sns.load_dataset('flights')
fli_df.head()

,year,month,passengers
0,1949,Jan,112
1,1949,Feb,118
2,1949,Mar,132
3,1949,Apr,129
4,1949,May,121


In [9]:
tip_df.to_csv('tips.csv', index=False)

In [10]:
fli_df.to_csv('flights.csv', index=False)

In [11]:
lazy_tip_df = pl.scan_csv('tips.csv')

In [25]:
lazy_fli_df = (pl.scan_csv('flights.csv'))
lazy_fli_select = lazy_fli_df.select(['year', 'month', 'passengers'])
lazy_fli_group = lazy_fli_select.group_by("year").agg([pl.col("passengers").sum().alias("count_passengers")]).sort('year').collect()

print(lazy_fli_group)

shape: (12, 2)
┌──────┬──────────────────┐
│ year ┆ count_passengers │
│ ---  ┆ ---              │
│ i64  ┆ i64              │
╞══════╪══════════════════╡
│ 1949 ┆ 1520             │
│ 1950 ┆ 1676             │
│ 1951 ┆ 2042             │
│ 1952 ┆ 2364             │
│ 1953 ┆ 2700             │
│ …    ┆ …                │
│ 1956 ┆ 3939             │
│ 1957 ┆ 4421             │
│ 1958 ┆ 4572             │
│ 1959 ┆ 5140             │
│ 1960 ┆ 5714             │
└──────┴──────────────────┘


In [44]:
fli_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   year        144 non-null    int64   
 1   month       144 non-null    category
 2   passengers  144 non-null    int64   
dtypes: category(1), int64(2)
memory usage: 2.9 KB


In [45]:
tip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [31]:
lazy_tip_df = (
    pl.scan_csv('tips.csv')
    .with_columns(pl.lit('1952').alias('year').cast(pl.Int64))
).collect()
print(lazy_tip_df)

shape: (244, 8)
┌────────────┬──────┬────────┬────────┬──────┬────────┬──────┬──────┐
│ total_bill ┆ tip  ┆ sex    ┆ smoker ┆ day  ┆ time   ┆ size ┆ year │
│ ---        ┆ ---  ┆ ---    ┆ ---    ┆ ---  ┆ ---    ┆ ---  ┆ ---  │
│ f64        ┆ f64  ┆ str    ┆ str    ┆ str  ┆ str    ┆ i64  ┆ i64  │
╞════════════╪══════╪════════╪════════╪══════╪════════╪══════╪══════╡
│ 16.99      ┆ 1.01 ┆ Female ┆ No     ┆ Sun  ┆ Dinner ┆ 2    ┆ 1952 │
│ 10.34      ┆ 1.66 ┆ Male   ┆ No     ┆ Sun  ┆ Dinner ┆ 3    ┆ 1952 │
│ 21.01      ┆ 3.5  ┆ Male   ┆ No     ┆ Sun  ┆ Dinner ┆ 3    ┆ 1952 │
│ 23.68      ┆ 3.31 ┆ Male   ┆ No     ┆ Sun  ┆ Dinner ┆ 2    ┆ 1952 │
│ 24.59      ┆ 3.61 ┆ Female ┆ No     ┆ Sun  ┆ Dinner ┆ 4    ┆ 1952 │
│ …          ┆ …    ┆ …      ┆ …      ┆ …    ┆ …      ┆ …    ┆ …    │
│ 29.03      ┆ 5.92 ┆ Male   ┆ No     ┆ Sat  ┆ Dinner ┆ 3    ┆ 1952 │
│ 27.18      ┆ 2.0  ┆ Female ┆ Yes    ┆ Sat  ┆ Dinner ┆ 2    ┆ 1952 │
│ 22.67      ┆ 2.0  ┆ Male   ┆ Yes    ┆ Sat  ┆ Dinner ┆ 2    ┆ 1952 │
│ 17

In [32]:
inner_joined_df = lazy_fli_group.join(lazy_tip_df, on='year', how='inner')
print(inner_joined_df)

Left Join Result:
 shape: (244, 9)
┌──────┬──────────────────┬────────────┬──────┬───┬────────┬──────┬────────┬──────┐
│ year ┆ count_passengers ┆ total_bill ┆ tip  ┆ … ┆ smoker ┆ day  ┆ time   ┆ size │
│ ---  ┆ ---              ┆ ---        ┆ ---  ┆   ┆ ---    ┆ ---  ┆ ---    ┆ ---  │
│ i64  ┆ i64              ┆ f64        ┆ f64  ┆   ┆ str    ┆ str  ┆ str    ┆ i64  │
╞══════╪══════════════════╪════════════╪══════╪═══╪════════╪══════╪════════╪══════╡
│ 1952 ┆ 2364             ┆ 16.99      ┆ 1.01 ┆ … ┆ No     ┆ Sun  ┆ Dinner ┆ 2    │
│ 1952 ┆ 2364             ┆ 10.34      ┆ 1.66 ┆ … ┆ No     ┆ Sun  ┆ Dinner ┆ 3    │
│ 1952 ┆ 2364             ┆ 21.01      ┆ 3.5  ┆ … ┆ No     ┆ Sun  ┆ Dinner ┆ 3    │
│ 1952 ┆ 2364             ┆ 23.68      ┆ 3.31 ┆ … ┆ No     ┆ Sun  ┆ Dinner ┆ 2    │
│ 1952 ┆ 2364             ┆ 24.59      ┆ 3.61 ┆ … ┆ No     ┆ Sun  ┆ Dinner ┆ 4    │
│ …    ┆ …                ┆ …          ┆ …    ┆ … ┆ …      ┆ …    ┆ …      ┆ …    │
│ 1952 ┆ 2364             ┆ 29.03      ┆ 

In [57]:
lazy_tip_df_dinner = (pl.scan_csv('tips.csv')
                    .filter(pl.col('time')=='Dinner')
                    .with_columns([pl.lit(None).alias('year').cast(pl.Int64),  
                    pl.lit('No data').alias('month').cast(pl.Utf8),  
                    pl.lit(None).alias('passengers').cast(pl.Int64)
]))

print(lazy_tip_df_dinner)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

 WITH_COLUMNS:
 [null.strict_cast(Int64).alias("year"), String(No data).alias("month"), null.strict_cast(Int64).alias("passengers")] 
  FILTER [(col("time")) == (String(Dinner))] FROM
    Csv SCAN [tips.csv]
    PROJECT */7 COLUMNS


In [58]:
lazy_fli_df_fifty = (pl.scan_csv('flights.csv').head(50).with_columns([
                    pl.lit(None).alias('total_bill').cast(pl.Float64),
                    pl.lit('No data').alias('sex').cast(pl.Utf8),  
                    pl.lit('No data').alias('smoker').cast(pl.Utf8),
                    pl.lit('No data').alias('day').cast(pl.Utf8),
                    pl.lit('No data').alias('time').cast(pl.Utf8),    
                    pl.lit(None).alias('tip').cast(pl.Float64),                    
                    pl.lit(None).alias('size').cast(pl.Int64), 
]))

print(lazy_fli_df_fifty)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

 WITH_COLUMNS:
 [null.strict_cast(Float64).alias("total_bill"), String(No data).alias("sex"), String(No data).alias("smoker"), String(No data).alias("day"), String(No data).alias("time"), null.strict_cast(Float64).alias("tip"), null.strict_cast(Int64).alias("size")] 
  SLICE[offset: 0, len: 50]
    Csv SCAN [flights.csv]
    PROJECT */3 COLUMNS


In [61]:
common_columns = sorted(lazy_tip_df_dinner.collect_schema().keys()) 
df1 = lazy_tip_df_dinner.collect().select(common_columns) 
df2 = lazy_fli_df_fifty.collect().select(common_columns) 


concat_df = pl.concat([df1, df2], how='vertical')


print(concat_df.head(10))

shape: (10, 10)
┌─────┬─────────┬────────────┬────────┬───┬────────┬──────┬────────────┬──────┐
│ day ┆ month   ┆ passengers ┆ sex    ┆ … ┆ time   ┆ tip  ┆ total_bill ┆ year │
│ --- ┆ ---     ┆ ---        ┆ ---    ┆   ┆ ---    ┆ ---  ┆ ---        ┆ ---  │
│ str ┆ str     ┆ i64        ┆ str    ┆   ┆ str    ┆ f64  ┆ f64        ┆ i64  │
╞═════╪═════════╪════════════╪════════╪═══╪════════╪══════╪════════════╪══════╡
│ Sun ┆ No data ┆ null       ┆ Female ┆ … ┆ Dinner ┆ 1.01 ┆ 16.99      ┆ null │
│ Sun ┆ No data ┆ null       ┆ Male   ┆ … ┆ Dinner ┆ 1.66 ┆ 10.34      ┆ null │
│ Sun ┆ No data ┆ null       ┆ Male   ┆ … ┆ Dinner ┆ 3.5  ┆ 21.01      ┆ null │
│ Sun ┆ No data ┆ null       ┆ Male   ┆ … ┆ Dinner ┆ 3.31 ┆ 23.68      ┆ null │
│ Sun ┆ No data ┆ null       ┆ Female ┆ … ┆ Dinner ┆ 3.61 ┆ 24.59      ┆ null │
│ Sun ┆ No data ┆ null       ┆ Male   ┆ … ┆ Dinner ┆ 4.71 ┆ 25.29      ┆ null │
│ Sun ┆ No data ┆ null       ┆ Male   ┆ … ┆ Dinner ┆ 2.0  ┆ 8.77       ┆ null │
│ Sun ┆ No data ┆ null  